# Transformación de Datos de Candidatos
Este notebook realiza las transformaciones necesarias sobre el dataset de candidatos, cargado previamente en la tabla `raw_candidates` de PostgreSQL, para generar la tabla final `applicant`. Las transformaciones incluyen la creación de una columna booleana `hired` basada en los criterios de puntajes, limpieza de datos y preparación para análisis o visualización en herramientas como Looker Studio.

---

## Configuración del Entorno
Aquí se importan las librerías necesarias y se configura la conexión con la base de datos PostgreSQL, reutilizando la configuración del notebook 1.

In [5]:
import pandas as pd
from sqlalchemy import create_engine
from dotenv import load_dotenv
import os

# Cargar variables de entorno
load_dotenv()
connection_string = f"postgresql://{os.getenv('DB_USER')}:{os.getenv('DB_PASSWORD')}@{os.getenv('DB_HOST')}:{os.getenv('DB_PORT')}/{os.getenv('DB_NAME')}"
engine = create_engine(connection_string)

# Verificar la conexión
print("Conexión a PostgreSQL establecida correctamente.")

Conexión a PostgreSQL establecida correctamente.


## Extracción de Datos
Cargamos los datos desde la tabla `raw_candidates` para realizar las transformaciones necesarias.

In [6]:
# Cargar datos desde raw_candidates
query = "SELECT * FROM raw_candidates;"
df = pd.read_sql(query, engine)

# Renombrar columnas para consistencia (mismo estilo que en el EDA)
df.rename(columns={
    'First Name': 'first_name',
    'Last Name': 'last_name',
    'Email': 'email',
    'Application Date': 'application_date',
    'Country': 'country',
    'YOE': 'years_of_experience',
    'Seniority': 'seniority',
    'Technology': 'technology',
    'Code Challenge Score': 'code_challenge_score',
    'Technical Interview Score': 'technical_interview_score'
}, inplace=True)

df['application_date'] = pd.to_datetime(df['application_date'], format='mixed')

print("Datos extraídos correctamente. Filas:", len(df))
df.head()

Datos extraídos correctamente. Filas: 50000


,first_name,last_name,email,application_date,country,years_of_experience,seniority,technology,code_challenge_score,technical_interview_score
0,Bernadette,Langworth,leonard91@yahoo.com,2021-02-26,Norway,2,Intern,Data Engineer,3,3
1,Camryn,Reynolds,zelda56@hotmail.com,2021-09-09,Panama,10,Intern,Data Engineer,2,10
2,Larue,Spinka,okey_schultz41@gmail.com,2020-04-14,Belarus,4,Mid-Level,Client Success,10,9
3,Arch,Spinka,elvera_kulas@yahoo.com,2020-10-01,Eritrea,25,Trainee,QA Manual,7,1
4,Larue,Altenwerth,minnie.gislason@gmail.com,2020-05-20,Myanmar,13,Mid-Level,Social Media Community Management,9,7


## Transformaciones de Datos
En esta sección, transformamos los datos para generar la tabla final `applicant`. Incluimos la creación de una columna booleana `hired` basada en los criterios identificados en el EDA (puntajes >= 7 en `code_challenge_score` y `technical_interview_score`), limpieza de duplicados por email (si aplica) y normalización de datos.

Nota:
Este código asume que, en un contexto real, los correos repetidos podrían indicar reaplicaciones, y mantenemos el registro más reciente (basado en application_date). Dado que los datos son ficticios, esta limpieza es opcional, pero alineada con el EDA.

In [ ]:
# Crear columna 'hired' basada en los criterios del EDA
df['hired'] = ((df['code_challenge_score'] >= 7) & (df['technical_interview_score'] >= 7)).astype(bool)

# Limpiar duplicados por email, manteniendo el registro más reciente (basado en application_date)
df = df.sort_values('application_date', ascending=False).drop_duplicates(subset='email', keep='first')

# Normalizar 'technology' y 'country'
# Por ejemplo, agrupar tecnologías similares o estandarizar nombres de países
df['technology'] = df['technology'].str.strip().str.title()
df['country'] = df['country'].str.strip().str.title()

# Verificar el resultado
print("Número de candidatos tras eliminar duplicados por email:", len(df))
print("Porcentaje de candidatos contratados:", (df['hired'].mean() * 100).round(2), "%")
df.head()

Número de candidatos tras eliminar duplicados por email: 49833
Porcentaje de candidatos contratados: 13.41 %


,first_name,last_name,email,application_date,country,years_of_experience,seniority,technology,code_challenge_score,technical_interview_score,hired
23098,Philip,Hessel,kallie.hilpert48@gmail.com,2022-07-04,Uzbekistan,28,Intern,Client Success,4,5,False
13850,Trystan,Crona,aditya.walsh@gmail.com,2022-07-04,Zambia,23,Intern,Adobe Experience Manager,2,4,False
13653,Clarissa,Rogahn,winston14@hotmail.com,2022-07-04,Brunei Darussalam,25,Architect,Qa Manual,7,0,False
45936,Maritza,Beer,theodora_grimes28@gmail.com,2022-07-04,Algeria,10,Architect,Data Engineer,6,8,False
38122,Eulalia,Schaden,chadd_wunsch@gmail.com,2022-07-04,Niger,0,Lead,Qa Manual,8,9,True


## Carga de la Tabla Final
Creamos la tabla `applicant` en PostgreSQL y cargamos los datos transformados. La tabla incluirá todas las columnas originales más la nueva columna `hired`.

In [8]:
try:
    df.to_sql('applicant', engine, if_exists='replace', index=False)
    print("Datos transformados cargados correctamente en la tabla applicant.")
except Exception as e:
    print(f"Error al cargar los datos en PostgreSQL: {e}")

Datos transformados cargados correctamente en la tabla applicant.


## Verificación de la Tabla Final
Confirmamos que los datos transformados se cargaron correctamente en la tabla `applicant` de PostgreSQL.

In [9]:
# Verificar los datos en applicant
query = "SELECT * FROM applicant LIMIT 5;"
final_df = pd.read_sql(query, engine)
print("Primeros 5 registros de la tabla applicant:")
final_df

Primeros 5 registros de la tabla applicant:


,first_name,last_name,email,application_date,country,years_of_experience,seniority,technology,code_challenge_score,technical_interview_score,hired
0,Philip,Hessel,kallie.hilpert48@gmail.com,2022-07-04,Uzbekistan,28,Intern,Client Success,4,5,False
1,Trystan,Crona,aditya.walsh@gmail.com,2022-07-04,Zambia,23,Intern,Adobe Experience Manager,2,4,False
2,Clarissa,Rogahn,winston14@hotmail.com,2022-07-04,Brunei Darussalam,25,Architect,Qa Manual,7,0,False
3,Maritza,Beer,theodora_grimes28@gmail.com,2022-07-04,Algeria,10,Architect,Data Engineer,6,8,False
4,Eulalia,Schaden,chadd_wunsch@gmail.com,2022-07-04,Niger,0,Lead,Qa Manual,8,9,True


## Conclusiones
Este notebook transformó los datos de la tabla `raw_candidates` para crear la tabla final `applicant`, añadiendo la columna `hired` basada en los criterios establecidos (puntajes >= 7 en ambos desafíos). Se eliminaron duplicados por email, manteniendo el registro más reciente, y se normalizaron algunas columnas para consistencia. Los datos están ahora listos para análisis o visualización desde herramientas externas.

Los próximos pasos incluyen generar visualizaciones o dashboards basados en esta tabla final.